# Tutoriel pytorch - TP3 - IFT725

Tel que mentionné dans l'énoncé du travail, vous devez recopier les blocs de code du tutoriel suivant

https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

en donnant, pour chaque bloc, une description en format "markdown" de son contenu.

# Warm-up: numpy
## Principales variables
- a : float64, premier poids aléatoire du modèle

- b : float64, deuxième poids aléatoire du modèle

- c : float64, troisième poids aléatoire du modèle

- d : float64, quatrième poids aléatoire du modèle

- grad_a : float64, gradient de a par rapport à la loss (dL/da), calculé lors de la backward pass

- grad_b : float64, gradient de b par rapport à la loss (dL/db), calculé lors de la backward pass

- grad_c : float64, gradient de c par rapport à la loss (dL/dc), calculé lors de la backward pass

- grad_d : float64, gradient de d par rapport à la loss (dL/dd), calculé lors de la backward pass

- grad_y_pred : ndarray à 1 dimension de taille 2000, gradient de y par rapport à la loss, dérivée de la loss par rapport à la prédiction (dL/dy), calculé lors de la backward pass

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- loss : float64, résultat de la fonction de perte 'square loss'

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : ndarray à 1 dimension de taille 2000, espace linéaire de -pi à pi, chaque point équidistant, variable indépendante et axe des abscisses

- y : ndarray à 1 dimension de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : ndarray à 1 dimension de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [ ]:
# -*- coding: utf-8 -*-
import numpy as np
import math

# Create random input and output data
x = np.linspace(-math.pi, math.pi, 2000)
y = np.sin(x)

# Randomly initialize weights
a = np.random.randn()
b = np.random.randn()
c = np.random.randn()
d = np.random.randn()

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    # y = a + b x + c x^2 + d x^3
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = np.square(y_pred - y).sum()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d

print(f'Result: y = {a} + {b} x + {c} x^2 + {d} x^3')

# PyTorch: Tensors
## Principales variables
- a : torch.float tensor, rang 0, premier poids aléatoire du modèle, envoyé vers le device spécifié

- b : torch.float tensor, rang 0, deuxième poids aléatoire du modèle, envoyé vers le device spécifié

- c : torch.float tensor, rang 0, troisième poids aléatoire du modèle, envoyé vers le device spécifié

- d : torch.float tensor, rang 0, quatrième poids aléatoire du modèle, envoyé vers le device spécifié

- device : torch.device, device vers où envoyer les tensors (CPU ou GPU). De base, les tensors sont envoyés vers le
CPU, sauf si on décommente la ligne qui envoie vers le GPU (cuda:0). cuda:0 veut dire le GPU à l'indice 0 lorsqu'on fait
la commande `nvidia-smi -L`.

- dtype : type de donnée spécifiée pour les tensors (torch.float)

- grad_a : torch.float tensor, rang 0, gradient de a par rapport à la loss (dL/da), calculé lors de la backward pass

- grad_b : torch.float tensor, rang 0, gradient de b par rapport à la loss (dL/db), calculé lors de la backward pass

- grad_c : torch.float tensor, rang 0, gradient de c par rapport à la loss (dL/dc), calculé lors de la backward pass

- grad_d : torch.float tensor, rang 0, gradient de d par rapport à la loss (dL/dd), calculé lors de la backward pass

- grad_y_pred : torch.float tensor, rang 1, de taille 2000, gradient de y par rapport à la loss, dérivée de la loss par
rapport à la prédiction (dL/dy), calculé lors de la backward pass

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- loss : float, résultat de la fonction de perte 'square loss'

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses, envoyé vers le device spécifié

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass


In [ ]:
# -*- coding: utf-8 -*-

import torch
import math


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

# Create random input and output data
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Randomly initialize weights
a = torch.randn((), device=device, dtype=dtype)
b = torch.randn((), device=device, dtype=dtype)
c = torch.randn((), device=device, dtype=dtype)
d = torch.randn((), device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum().item()
    if t % 100 == 99:
        print(t, loss)

    # Backprop to compute gradients of a, b, c, d with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_a = grad_y_pred.sum()
    grad_b = (grad_y_pred * x).sum()
    grad_c = (grad_y_pred * x ** 2).sum()
    grad_d = (grad_y_pred * x ** 3).sum()

    # Update weights using gradient descent
    a -= learning_rate * grad_a
    b -= learning_rate * grad_b
    c -= learning_rate * grad_c
    d -= learning_rate * grad_d


print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

# PyTorch: Tensors and autograd
## Principales variables
- a : torch.float tensor, rang 0, premier poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass

- b : torch.float tensor, rang 0, deuxième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass

- c : torch.float tensor, rang 0, troisième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass

- d : torch.float tensor, rang 0, quatrième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass

- device : torch.device, device vers où envoyer les tensors (CPU ou GPU). De base, les tensors sont envoyés vers le
CPU, sauf si on décommente la ligne qui envoie vers le GPU (cuda:0). cuda:0 veut dire le GPU à l'indice 0 lorsqu'on fait
la commande `nvidia-smi -L`.

- dtype : type de donnée spécifiée pour les tensors (torch.float)

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- loss : torch.float tensor, rang 0, résultat de la fonction de perte 'square loss', fonction de gradient grad_fn=
<SumBackward0>

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses, envoyé vers le device spécifié

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [1]:
# -*- coding: utf-8 -*-
import torch
import math

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For a third order polynomial, we need
# 4 weights: y = a + b x + c x^2 + d x^3
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.randn((), device=device, dtype=dtype, requires_grad=True)
b = torch.randn((), device=device, dtype=dtype, requires_grad=True)
c = torch.randn((), device=device, dtype=dtype, requires_grad=True)
d = torch.randn((), device=device, dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
    # Forward pass: compute predicted y using operations on Tensors.
    y_pred = a + b * x + c * x ** 2 + d * x ** 3

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call a.grad, b.grad. c.grad and d.grad will be Tensors holding
    # the gradient of the loss with respect to a, b, c, d respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')


99 1137.969970703125
199 792.7483520507812
299 553.5726318359375
399 387.7051086425781
499 272.5675048828125
599 192.57078552246094
699 136.93980407714844
799 98.21990203857422
899 71.24766540527344
999 52.44377517700195
1099 39.324275970458984
1199 30.164026260375977
1299 23.763568878173828
1399 19.288406372070312
1499 16.15732192993164
1599 13.965250015258789
1699 12.429679870605469
1799 11.353375434875488
1899 10.598555564880371
1999 10.068940162658691
Result: y = 0.035517241805791855 + 0.8458353877067566 x + -0.006127315107733011 x^2 + -0.0917791798710823 x^3


# PyTorch: Defining new autograd functions
## Principales variables
- LegendrePolynomial3 : classe enfant qui hérite de la classe torch.autograd.Function,
fonction d'autograd avec nouvelles implémentations de forward et backward pass
basées sur les polynômes de Legendre

- P3 : builtin_function_or_method, alias pour Function.apply

- a : torch.float tensor, rang 0, premier poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass, initialisé à 0.0

- b : torch.float tensor, rang 0, deuxième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass, initialisé à -1.0

- c : torch.float tensor, rang 0, troisième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass, initialisé à 0.0

- d : torch.float tensor, rang 0, quatrième poids aléatoire du modèle, envoyé vers le device spécifié, requires_grad =
True pour calculer le gradient de ce tensor durant la backward pass, initialisé à 0.3

- device : torch.device, device vers où envoyer les tensors (CPU ou GPU). De base, les tensors sont envoyés vers le
CPU, sauf si on décommente la ligne qui envoie vers le GPU (cuda:0). cuda:0 veut dire le GPU à l'indice 0 lorsqu'on fait
la commande `nvidia-smi -L`.

- dtype : type de donnée spécifiée pour les tensors (torch.float)

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 5e-6

- loss : torch.float tensor, rang 0, résultat de la fonction de perte 'square loss', fonction de gradient grad_fn=
<SumBackward0>

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses, envoyé vers le device spécifié

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [2]:
# -*- coding: utf-8 -*-
import torch
import math


class LegendrePolynomial3(torch.autograd.Function):
    """
    We can implement our own custom autograd Functions by subclassing
    torch.autograd.Function and implementing the forward and backward passes
    which operate on Tensors.
    """

    @staticmethod
    def forward(ctx, input):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        ctx.save_for_backward(input)
        return 0.5 * (5 * input ** 3 - 3 * input)

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        input, = ctx.saved_tensors
        return grad_output * 1.5 * (5 * input ** 2 - 1)


dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0")  # Uncomment this to run on GPU

# Create Tensors to hold input and outputs.
# By default, requires_grad=False, which indicates that we do not need to
# compute gradients with respect to these Tensors during the backward pass.
x = torch.linspace(-math.pi, math.pi, 2000, device=device, dtype=dtype)
y = torch.sin(x)

# Create random Tensors for weights. For this example, we need
# 4 weights: y = a + b * P3(c + d * x), these weights need to be initialized
# not too far from the correct result to ensure convergence.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
a = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
b = torch.full((), -1.0, device=device, dtype=dtype, requires_grad=True)
c = torch.full((), 0.0, device=device, dtype=dtype, requires_grad=True)
d = torch.full((), 0.3, device=device, dtype=dtype, requires_grad=True)

learning_rate = 5e-6
for t in range(2000):
    # To apply our Function, we use Function.apply method. We alias this as 'P3'.
    P3 = LegendrePolynomial3.apply

    # Forward pass: compute predicted y using operations; we compute
    # P3 using our custom autograd operation.
    y_pred = a + b * P3(c + d * x)

    # Compute and print loss
    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())

    # Use autograd to compute the backward pass.
    loss.backward()

    # Update weights using gradient descent
    with torch.no_grad():
        a -= learning_rate * a.grad
        b -= learning_rate * b.grad
        c -= learning_rate * c.grad
        d -= learning_rate * d.grad

        # Manually zero the gradients after updating weights
        a.grad = None
        b.grad = None
        c.grad = None
        d.grad = None

print(f'Result: y = {a.item()} + {b.item()} * P3({c.item()} + {d.item()} x)')


99 209.9583282470703
199 144.6602020263672
299 100.7025146484375
399 71.03520965576172
499 50.97850799560547
599 37.40315246582031
699 28.20688247680664
799 21.97319221496582
899 17.7457275390625
999 14.877889633178711
1099 12.93176555633545
1199 11.610918998718262
1299 10.71424674987793
1399 10.105476379394531
1499 9.69210433959961
1599 9.411375045776367
1699 9.220744132995605
1799 9.091285705566406
1899 9.003360748291016
1999 8.943641662597656
Result: y = 3.5881797533221516e-09 + -2.208526849746704 * P3(-1.6777875755380478e-09 + 0.2554861009120941 x)


# PyTorch: nn
## Principales variables

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 1e-6

- linear_layer : torch.nn.Linear, première couche du modèle, 3 features en entrée et
1 feature en sortie, avec biais

- loss : torch.float tensor, rang 0, résultat de la fonction de perte loss_fn = torch.nn.MSELoss(reduction='sum'),
fonction de gradient grad_fn=<MseLossBackward>

- loss_fn : torch.nn.MSELoss, fonction de perte 'Mean Square Error (MSE)' mais avec la reduction='sum' donc enlevant
la division par n, donc devient une fonction de perte 'Square Error'

- model : torch.nn.Sequential, modèle de réseau de neurones combiné d'une première couche
linéaire avec 3 features d'entrée et 1 feature en sortie, suivi d'une deuxième couche
d'aplatissement qui aplatit la sortie de la première couche vers un tensor 1D
de la même taille que y

- p : torch.int64 tensor, rang 1, de taille 3, formé des entiers 1, 2 et 3 qui
sont les exposants à appliquer au tenseur d'entrée

- param : parameter, variable qui itère à travers la liste des paramètres dans le modèle
lors de la descente de gradient

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses

- xx : torch.float tensor, rang 2, de taille 2000x3, qui représente (x, x^2, x^3),
données d'entrée pour la forward pass

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [1]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# For this example, the output y is a linear function of (x, x^2, x^3), so
# we can consider it as a linear layer neural network. Let's prepare the
# tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# In the above code, x.unsqueeze(-1) has shape (2000, 1), and p has shape
# (3,), for this case, broadcasting semantics will apply to obtain a tensor
# of shape (2000, 3)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. The Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
# The Flatten layer flatens the output of the linear layer to a 1D tensor,
# to match the shape of `y`.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
for t in range(2000):

    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(xx)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero the gradients before running the backward pass.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

# You can access the first layer of `model` like accessing the first item of a list
linear_layer = model[0]

# For linear layer, its parameters are stored as `weight` and `bias`.
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')

99 1262.0028076171875
199 842.3462524414062
299 563.4276733398438
399 377.9873352050781
499 254.6532440185547
599 172.59512329101562
699 117.97789001464844
799 81.6100845336914
899 57.3834342956543
999 41.23725128173828
1099 30.47127342224121
1199 23.28896713256836
1299 18.494962692260742
1399 15.293288230895996
1499 13.153756141662598
1599 11.723140716552734
1699 10.76589298248291
1799 10.124978065490723
1899 9.695557594299316
1999 9.407618522644043
Result: y = 0.012823732569813728 + 0.8362652063369751 x + -0.002212307881563902 x^2 + -0.09041790664196014 x^3


# PyTorch: optim
## Principales variables

- learning_rate : float, taux d'apprentissage pour la descente du gradient, fixé à 0.001

- linear_layer : torch.nn.Linear, première couche du modèle, 3 features en entrée et
1 feature en sortie, avec biais

- loss : torch.float tensor, rang 0, résultat de la fonction de perte loss_fn = torch.nn.MSELoss(reduction='sum'),
fonction de gradient grad_fn=<MseLossBackward>

- loss_fn : torch.nn.MSELoss, fonction de perte 'Mean Square Error (MSE)' mais avec la reduction='sum' donc enlevant
la division par n, donc devient une fonction de perte 'Square Error'

- model : torch.nn.Sequential, modèle de réseau de neurones combiné d'une première couche
linéaire avec 3 features d'entrée et 1 feature en sortie, suivi d'une deuxième couche
d'aplatissement qui aplatit la sortie de la première couche vers un tensor 1D
de la même taille que y

- optimizer : torch.optim.RMSprop, optimisateur qui met automatiquement à jour les poids du modèle
durant l'entraînement, de type RMSprop avec un learning rate de 0.001

- p : torch.int64 tensor, rang 1, de taille 3, formé des entiers 1, 2 et 3 qui
sont les exposants à appliquer au tenseur d'entrée

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses

- xx : torch.float tensor, rang 2, de taille 2000x3, qui représente (x, x^2, x^3),
données d'entrée pour la forward pass

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [1]:
# -*- coding: utf-8 -*-
import torch
import math


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Prepare the input tensor (x, x^2, x^3).
p = torch.tensor([1, 2, 3])
xx = x.unsqueeze(-1).pow(p)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(3, 1),
    torch.nn.Flatten(0, 1)
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use RMSprop; the optim package contains many other
# optimization algorithms. The first argument to the RMSprop constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
for t in range(2000):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(xx)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()


linear_layer = model[0]
print(f'Result: y = {linear_layer.bias.item()} + {linear_layer.weight[:, 0].item()} x + {linear_layer.weight[:, 1].item()} x^2 + {linear_layer.weight[:, 2].item()} x^3')


99 6818.57421875
199 2218.654052734375
299 1109.0950927734375
399 919.4545288085938
499 827.8740234375
599 721.5701904296875
699 602.0877075195312
799 481.0479736328125
899 369.1165466308594
999 271.884765625
1099 191.18289184570312
1199 127.14825439453125
1299 78.80008697509766
1399 45.151756286621094
1499 24.298276901245117
1599 13.718693733215332
1699 9.897330284118652
1799 8.980656623840332
1899 8.884114265441895
1999 8.900309562683105
Result: y = 0.0004896934842690825 + 0.8563463687896729 x + 0.0004897221806459129 x^2 + -0.09378717094659805 x^3


# PyTorch: Custom nn Modules
## Principales variables

- criterion : torch.nn.MSELoss, fonction de perte 'Mean Square Error (MSE)' mais avec la reduction='sum' donc enlevant
la division par n, donc devient une fonction de perte 'Square Error'

- loss : torch.float tensor, rang 0, résultat de la fonction de perte loss_fn = torch.nn.MSELoss(reduction='sum'),
fonction de gradient grad_fn=<MseLossBackward>

- model : instance de la classe Polynomial3, cette classe enfant hérite de la classe torch.nn.Module avec
quatre attributs de plus définis : a, b, c et d qui sont des nouveaux paramètres
créés à partir de nombres aléatoires. La méthode forward est redéfinie pour retourner
a + b*x + c*x^2 + d*x^3. La méthode string retourne la string "y = a + b x + c x^2 + d x^3"

- optimizer : torch.optim.SGD, optimisateur qui met automatiquement à jour les poids du modèle
durant l'entraînement, de type SGD (Stochastic Gradient Descent) avec un learning rate de 1e-6

- t : int, boucle de 0 à 1999, compteur d'itération dans la boucle d'apprentissage

- x : torch.float tensor, rang 1, de taille 2000, espace linéaire de -pi à pi, chaque point équidistant,
variable indépendante et axe des abscisses

- y : torch.float tensor, rang 1, de taille 2000, vérité terrain pour la fonction sin(x)

- y_pred : torch.float tensor, rang 1, de taille 2000, prédiction du modèle pour la fonction sin(x), résultat de la
forward pass

In [1]:
# -*- coding: utf-8 -*-
import torch
import math


class Polynomial3(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate four parameters and assign them as
        member parameters.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        In the forward function we accept a Tensor of input data and we must return
        a Tensor of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Tensors.
        """
        return self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = Polynomial3()

# Construct our loss function and an Optimizer. The call to model.parameters()
# in the SGD constructor will contain the learnable parameters of the nn.Linear
# module which is members of the model.
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-6)
for t in range(2000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')

99 337.92010498046875
199 235.30039978027344
299 164.8531951904297
399 116.43585968017578
499 83.12185668945312
599 60.17447280883789
699 44.3505859375
799 33.42705154418945
899 25.878379821777344
999 20.656429290771484
1099 17.04045295715332
1199 14.534031867980957
1299 12.795023918151855
1399 11.5873384475708
1499 10.747886657714844
1599 10.163878440856934
1699 9.757232666015625
1799 9.473849296569824
1899 9.276211738586426
1999 9.138267517089844
Result: y = -0.01732071116566658 + 0.8496417999267578 x + 0.002988110762089491 x^2 + -0.09232060611248016 x^3


# PyTorch: Control Flow + Weight Sharing
## Description


In [ ]:
# -*- coding: utf-8 -*-
import random
import torch
import math


class DynamicNet(torch.nn.Module):
    def __init__(self):
        """
        In the constructor we instantiate five parameters and assign them as members.
        """
        super().__init__()
        self.a = torch.nn.Parameter(torch.randn(()))
        self.b = torch.nn.Parameter(torch.randn(()))
        self.c = torch.nn.Parameter(torch.randn(()))
        self.d = torch.nn.Parameter(torch.randn(()))
        self.e = torch.nn.Parameter(torch.randn(()))

    def forward(self, x):
        """
        For the forward pass of the model, we randomly choose either 4, 5
        and reuse the e parameter to compute the contribution of these orders.

        Since each forward pass builds a dynamic computation graph, we can use normal
        Python control-flow operators like loops or conditional statements when
        defining the forward pass of the model.

        Here we also see that it is perfectly safe to reuse the same parameter many
        times when defining a computational graph.
        """
        y = self.a + self.b * x + self.c * x ** 2 + self.d * x ** 3
        for exp in range(4, random.randint(4, 6)):
            y = y + self.e * x ** exp
        return y

    def string(self):
        """
        Just like any class in Python, you can also define custom method on PyTorch modules
        """
        return f'y = {self.a.item()} + {self.b.item()} x + {self.c.item()} x^2 + {self.d.item()} x^3 + {self.e.item()} x^4 ? + {self.e.item()} x^5 ?'


# Create Tensors to hold input and outputs.
x = torch.linspace(-math.pi, math.pi, 2000)
y = torch.sin(x)

# Construct our model by instantiating the class defined above
model = DynamicNet()

# Construct our loss function and an Optimizer. Training this strange model with
# vanilla stochastic gradient descent is tough, so we use momentum
criterion = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-8, momentum=0.9)
for t in range(30000):
    # Forward pass: Compute predicted y by passing x to the model
    y_pred = model(x)

    # Compute and print loss
    loss = criterion(y_pred, y)
    if t % 2000 == 1999:
        print(t, loss.item())

    # Zero gradients, perform a backward pass, and update the weights.
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(f'Result: {model.string()}')